#### Coding interview question:   
Write a Go function to merge multiple input channels into one output channel.   
How do you ensure no data loss and proper closure?   
Fan-in patterns like this are a staple of distributed systems and concurrent code!    
Handling multiple streams, guaranteeing no value loss, and clean closure is tougher than it looks.

In [2]:
func generate[T any](nums ...T) <-chan T {
	out := make(chan T)
	go func() {
		for _, n := range nums {
			out <- n
		}
		close(out)
	}()
	return out
}

func merge[T any](chs ...<-chan T) <-chan T {
	out := make(chan T)
	var wg sync.WaitGroup
	wg.Add(len(chs))
	for _, ch := range chs {
		go func(c <-chan T) {
			for v := range c {
				out <- v
			}
			wg.Done()
		}(ch)
	}
	go func() {
		wg.Wait()
		close(out)
	}()
	return out
}

func main() {
	ch1 := generate(1, 2, 3, 4, 5, 6)
	ch2 := generate(21, 22, 23, 24, 25, 26)
	ch3 := generate(31, 32, 33, 34, 35, 36)
	ch4 := generate(41, 42, 43, 44, 45, 46)
	for data := range merge(ch1, ch2, ch3, ch4) {
		fmt.Println(data)
	}
}

1
21
31
41
2
3
22
32
42
43
4
23
33
44
45
5
24
34
35
46
6
25
36
26
